In [1]:
import collections
import dataclasses as dcls
import functools
import math
import os
import string
from typing import Tuple, List, Dict, Any, Optional, Callable

from keras_applications.imagenet_utils import _obtain_input_shape
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
tf.__version__, tf.keras.__version__, tfa.__version__

from sds.model.params import ScaledParams
from sds.utils.utils import compose


2022-01-27 21:52:25.915769: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
MOMENTUM = 0.997
EPSILON = 1e-4


In [3]:
phi = 0
num_classes = 20
freeze_bn = True
params = ScaledParams(phi)
image_input = tf.keras.Input(params.input_shape)
_, bb_feature_maps = params.backbone_class(input_tensor=image_input, freeze_bn=freeze_bn)

2022-01-27 21:52:28.547792: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-27 21:52:28.549043: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-27 21:52:28.571628: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-01-27 21:52:28.571663: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: train
2022-01-27 21:52:28.571672: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: train
2022-01-27 21:52:28.571775: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.86.0
2022-01-27 21:52:28.571801: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.86.0
2022-01-27 21:52:28.571809: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] ker

In [4]:
print(params.__dict__)
for bb_feature_map in bb_feature_maps:
    print(bb_feature_map)

{'phi': 0, 'input_size': 512, 'input_shape': (512, 512, 3), 'bifpn_width': 64, 'bifpn_depth': 3, 'subnet_width': 64, 'subnet_depth': 3, 'subnet_num_iteration_steps': 1, 'num_groups_gn': 4, 'backbone_class': <function EfficientNetB0 at 0x7efe7fca84c0>}
KerasTensor(type_spec=TensorSpec(shape=(None, 256, 256, 16), dtype=tf.float32, name=None), name='block1b_project_bn/FusedBatchNormV3:0', description="created by layer 'block1b_project_bn'")
KerasTensor(type_spec=TensorSpec(shape=(None, 128, 128, 24), dtype=tf.float32, name=None), name='block2c_add/add:0', description="created by layer 'block2c_add'")
KerasTensor(type_spec=TensorSpec(shape=(None, 64, 64, 40), dtype=tf.float32, name=None), name='block3c_add/add:0', description="created by layer 'block3c_add'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 112), dtype=tf.float32, name=None), name='block5d_add/add:0', description="created by layer 'block5d_add'")
KerasTensor(type_spec=TensorSpec(shape=(None, 16, 16, 320), dtype=tf.flo

In [9]:
def prefixer(prefix: str = ''):
    if not prefix:
        return lambda name: name
    return lambda name: f'{prefix}/{name}'


def bifpn_init(feature_maps: List[Any], num_channels: int, name: str) -> List[Any]:
    pname = prefixer(name)
    res = []
    for i, features_in in enumerate(feature_maps):
        features_out = tf.keras.layers.SeparableConv2D(
            num_channels, kernel_size=3, strides=1, padding='same', name=pname(f'SeparableConv2D{i}'))(features_in)
        res.append(features_out)
    return res


def SeparableConvBlock(num_channels: int, kernel_size: int, strides: int, gn_enabled: bool, name: str):
    pname = prefixer(name)
    f1 = tf.keras.layers.SeparableConv2D(num_channels, kernel_size=kernel_size, strides=strides, padding='same',
        use_bias=True, name=pname('SeparableConv2D'))
    if not gn_enabled:
        return f1
    
    f2 = tfa.layers.GroupNormalization(groups=num_channels // 16)
    return compose(f1, f2)


class BifpnWeightedAdd(tf.keras.layers.Layer):
    def __init__(self, epsilon=1e-4, **kwargs):
        super().__init__(**kwargs)
        self.epsilon = epsilon
        self.w = None

    def build(self, input_shape):
        num_in = len(input_shape)
        self.w = self.add_weight(name='w',
                                 shape=(num_in,),
                                 initializer=tf.keras.initializers.constant(1 / num_in),
                                 trainable=True,
                                 dtype=tf.float32)

    def call(self, inputs, **kwargs):
        w = tf.keras.activations.relu(self.w)
        x = tf.reduce_sum([w[i] * inputs[i] for i in range(len(inputs))], axis=0)
        x = x / (tf.reduce_sum(w) + self.epsilon)
        return x

    def compute_output_shape(self, input_shape):
        # Input is a list of similar shapes: [TensorShape[m, n], TensorShape[m, n], ...]
        return input_shape[0]

    def get_config(self):
        config = {
            **super().get_config(),
            'epsilon': self.epsilon,
        }
        return config


def bifpn_merge(feature_maps_cur_level: List[Any], feature_map_other_level, upsample_other: bool, num_channels: int,
        gn_enabled: bool, name: str):
    pname = prefixer(name)
    if upsample_other:
        feature_map_resampled = tf.keras.layers.UpSampling2D(name=pname('UpSampling2D'))(feature_map_other_level)
    else:
        feature_map_resampled = tf.keras.layers.MaxPooling2D(
            pool_size=3, strides=2, padding='same', name=pname('MaxPooling2D'))(feature_map_other_level)
    
    feature_map = BifpnWeightedAdd(name=pname('BifpnWeightedAdd'))(feature_maps_cur_level + [feature_map_resampled])
    feature_map = tf.keras.activations.swish(feature_map)
    feature_map = SeparableConvBlock(
        num_channels=num_channels, kernel_size=3, strides=1, gn_enabled=gn_enabled, name=pname('SeparableConvBlock'))(feature_map)
    return feature_map


def bifpn_top_down(feature_maps: List[Any], num_channels: int, gn_enabled: bool, name: str) -> List[Any]:
    pname = prefixer(name)
    features_out = [feature_maps[0]]
    for i in range(1, len(feature_maps)):
        features_merged = bifpn_merge(
            feature_maps_cur_level=[feature_maps[i]],
            feature_map_other_level=features_out[-1],
            upsample_other=True,
            num_channels=num_channels,
            gn_enabled=gn_enabled,
            name=pname(f'merge{i}'))
        features_out.append(features_merged)
    return features_out


def bifpn_bottom_up(feature_maps: List[List[Any]], num_channels: int, gn_enabled: bool, name: str) -> List[Any]:
    pname = prefixer(name)
    features_out = [feature_maps[0][0]]
    for i in range(1, len(feature_maps)):
        features_merged = bifpn_merge(
            feature_maps_cur_level=feature_maps[i],
            feature_map_other_level=features_out[-1],
            upsample_other=False,
            num_channels=num_channels,
            gn_enabled=gn_enabled,
            name=pname(f'merge{i}'),
        )
        features_out.append(features_merged)
    return features_out
    


def bifpn_layer(feature_maps: List[Any], num_channels: int, gn_enabled: bool = True, name: str = 'BifpnLayer') -> List[Any]:
    pname = prefixer(name)
    feature_maps = list(reversed(feature_maps))
    features_top_down = bifpn_top_down(feature_maps, num_channels, gn_enabled=gn_enabled, name=pname('BifpnTopDown'))
    features_mid = []
    n_maps = len(feature_maps)
    for i in range(n_maps):
        if 0 < i < n_maps - 1:
            features_mid.append([feature_maps[i], features_top_down[i]])
        else:
            features_mid.append([features_top_down[i]])
    
    features_mid = list(reversed(features_mid))
    features_bottom_up = bifpn_bottom_up(features_mid, num_channels, gn_enabled=gn_enabled, name=pname('BifpnBottomUp'))
    
    return features_bottom_up


In [10]:
fpn_init_feature_maps = bifpn_init(bb_feature_maps, params.bifpn_width, name='BifpnInit')
fpn_feature_maps = bifpn_layer(fpn_init_feature_maps, params.bifpn_width, name='Bifpn1')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [11]:
print(fpn_feature_maps)

[<KerasTensor: shape=(None, 256, 256, 64) dtype=float32 (created by layer 'group_normalization_11')>, <KerasTensor: shape=(None, 128, 128, 64) dtype=float32 (created by layer 'group_normalization_12')>, <KerasTensor: shape=(None, 64, 64, 64) dtype=float32 (created by layer 'group_normalization_13')>, <KerasTensor: shape=(None, 32, 32, 64) dtype=float32 (created by layer 'group_normalization_14')>, <KerasTensor: shape=(None, 16, 16, 64) dtype=float32 (created by layer 'group_normalization_15')>]


In [15]:
def final_upscale(fpn_feature_maps: List[Any], num_channels: int, name: str):
    pname = prefixer(name)
    feature_maps_in = []
    for i, feature_map in enumerate(reversed(fpn_feature_maps)):
        feature_map_in = tf.keras.layers.SeparableConv2D(
            num_channels, kernel_size=3, strides=1, padding='same', name=pname(f'SeparableConv2D{i}'))(feature_map)
        feature_maps_in.append(feature_map_in)
    
    feature_maps = bifpn_top_down(feature_maps_in, num_channels=num_channels, gn_enabled=False, name=pname('BifpnTopDown'))
    feature_map_out = feature_maps[-1]
    feature_map_out = tf.keras.layers.Conv2DTranspose(num_channels, kernel_size=3, strides=2, padding='same')(feature_map_out)
    return feature_map_out


In [16]:
n_classes = 28
n_channels_normals = 3
n_channels_cmap = 1
n_channels_contours = 1
n_channels_out = n_classes + n_channels_normals + n_channels_cmap + n_channels_contours
print(f'Ouput channels: {n_channels_out}')
features_out = final_upscale(fpn_feature_maps, n_channels_out, name='FinalUpscale')

Ouput channels: 33


In [20]:
model = tf.keras.models.Model(inputs=[image_input], outputs=features_out)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 256, 256, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 256, 256, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 256, 256, 32) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [28]:
BN1.moving_mean, BN2.moving_mean

(<tf.Variable 'batch_normalization_9/moving_mean:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization_10/moving_mean:0' shape=(2,) dtype=float32, numpy=array([1., 1.], dtype=float32)>)

In [8]:
import sys
sys.path

['/home/misha/prog/sixd_sense/sds',
 '/home/misha/.vscode-insiders/extensions/ms-toolsai.jupyter-2022.1.1001614873/pythonFiles',
 '/home/misha/.vscode-insiders/extensions/ms-toolsai.jupyter-2022.1.1001614873/pythonFiles/lib/python',
 '/home/misha/anaconda3/envs/bop/lib/python39.zip',
 '/home/misha/anaconda3/envs/bop/lib/python3.9',
 '/home/misha/anaconda3/envs/bop/lib/python3.9/lib-dynload',
 '',
 '/home/misha/anaconda3/envs/bop/lib/python3.9/site-packages',
 '/home/misha/prog/lib/BlenderProc',
 '/home/misha/anaconda3/envs/bop/lib/python3.9/site-packages/IPython/extensions',
 '/home/misha/.ipython',
 '/home/misha/.local/lib/python3.9/site-packages']

In [43]:
x = tf.random.uniform((5, 2), 1, 20, tf.int32)
x = tf.cast(x, tf.float32)
x

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[16., 12.],
       [19.,  9.],
       [ 4., 11.],
       [ 5.,  2.],
       [17., 11.]], dtype=float32)>

In [60]:
dp = tf.keras.layers.Dropout(0.2, (None, 1))
dp

In [63]:
dp(x, training=True)

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[20.  , 15.  ],
       [23.75, 11.25],
       [ 0.  ,  0.  ],
       [ 6.25,  2.5 ],
       [ 0.  ,  0.  ]], dtype=float32)>

In [65]:
i = tf.keras.Input((None, 2, 3))
type(i)

tensorflow.python.keras.engine.keras_tensor.KerasTensor

In [68]:
i.__class__.__bases__

(object,)

In [69]:
type(dp(x, training=True))

tensorflow.python.framework.ops.EagerTensor

In [5]:
ba = DEFAULT_BLOCKS_ARGS[0]
ba

BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, strides=(1, 1), se_ratio=0.25)

In [12]:
BlockArgs()

TypeError: __main__.BlockArgs() got multiple values for keyword argument 'num_repeat'

In [13]:
{**vars(ba), 'num_repeat': 2}

{'kernel_size': 3,
 'num_repeat': 2,
 'input_filters': 32,
 'output_filters': 16,
 'expand_ratio': 1,
 'id_skip': True,
 'strides': (1, 1),
 'se_ratio': 0.25}

In [15]:
from copy import copy
ba1 = copy(ba)
ba1.num_repeat = 22
ba, ba1

(BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, strides=(1, 1), se_ratio=0.25),
 BlockArgs(kernel_size=3, num_repeat=22, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, strides=(1, 1), se_ratio=0.25))

In [32]:
params = tf.constant([[['a0', 'b0'], ['c0', 'd0']],
              [['a1', 'b1'], ['c1', 'd1']]])
indices = [[[1, 0]], [[0, 1]]]
tf.gather_nd(params, indices), tf.gather_nd(params, indices, batch_dims=1)

(<tf.Tensor: shape=(2, 1, 2), dtype=string, numpy=
 array([[[b'a1', b'b1']],
 
        [[b'c0', b'd0']]], dtype=object)>,
 <tf.Tensor: shape=(2, 1), dtype=string, numpy=
 array([[b'c0'],
        [b'b1']], dtype=object)>)

In [20]:
ba1 = dcls.replace(ba, num_repeat=3)

In [21]:
ba, ba1

(BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, strides=(1, 1), se_ratio=0.25),
 BlockArgs(kernel_size=3, num_repeat=3, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, strides=(1, 1), se_ratio=0.25))

In [22]:
block_args = DEFAULT_BLOCKS_ARGS[0]
print(block_args)
width_coefficient, depth_coefficient, depth_divisor = 1.2, 1.4, 8
dcls.replace(block_args,
            input_filters=round_filters(block_args.input_filters, width_coefficient, depth_divisor),
            output_filters=round_filters(block_args.output_filters, width_coefficient, depth_divisor),
            num_repeat=round_repeats(block_args.num_repeat, depth_coefficient))


BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16, expand_ratio=1, id_skip=True, strides=(1, 1), se_ratio=0.25)


BlockArgs(kernel_size=3, num_repeat=2, input_filters=40, output_filters=24, expand_ratio=1, id_skip=True, strides=(1, 1), se_ratio=0.25)

In [2]:
t = tf.random.uniform((2, 3, 3, 4))
t

2022-01-09 23:51:06.058539: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-09 23:51:06.059416: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-09 23:51:06.086014: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-01-09 23:51:06.086047: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: train
2022-01-09 23:51:06.086054: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: train
2022-01-09 23:51:06.086138: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.86.0
2022-01-09 23:51:06.086161: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.86.0
2022-01-09 23:51:06.086166: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] ker

<tf.Tensor: shape=(2, 3, 3, 4), dtype=float32, numpy=
array([[[[0.4745221 , 0.11489785, 0.12162721, 0.07320273],
         [0.9412204 , 0.7219523 , 0.9624729 , 0.0850718 ],
         [0.9367769 , 0.5110214 , 0.83429956, 0.5570532 ]],

        [[0.4119432 , 0.575389  , 0.7608855 , 0.33425868],
         [0.23874879, 0.34591877, 0.4665196 , 0.5286418 ],
         [0.0789181 , 0.7882943 , 0.16099072, 0.41588652]],

        [[0.04509509, 0.0629158 , 0.9308244 , 0.47065175],
         [0.43653476, 0.9777608 , 0.81097436, 0.4424522 ],
         [0.94104016, 0.05488026, 0.95357215, 0.7789507 ]]],


       [[[0.02088416, 0.37490594, 0.49665165, 0.6692965 ],
         [0.49432635, 0.93393123, 0.4224975 , 0.40523756],
         [0.62095666, 0.03527641, 0.31722033, 0.07540357]],

        [[0.99739444, 0.09169388, 0.97013843, 0.8367541 ],
         [0.9673866 , 0.52303195, 0.23018289, 0.02071369],
         [0.5157801 , 0.42481792, 0.6533141 , 0.03869045]],

        [[0.77005005, 0.05051482, 0.5765488 , 0.4

In [4]:
t1 = tf.keras.activations.softmax(t)

In [17]:
tf.expand_dims(1, axis=0)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([1], dtype=int32)>

In [9]:
np.allclose(t2, np.ones(t2.get_shape()))

True

In [18]:
init = tf.keras.initializers.random_uniform(0, 10)

In [22]:
t = init((2, 2, 3))
t

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[4.4560995e+00, 2.6941299e-03, 8.4572754e+00],
        [8.5883799e+00, 6.0469198e+00, 2.9005849e+00]],

       [[3.6264551e+00, 6.6117930e+00, 8.4459186e-01],
        [5.9046197e+00, 6.6545572e+00, 1.9151568e+00]]], dtype=float32)>

In [24]:
t[..., :2]

<tf.Tensor: shape=(2, 2, 2), dtype=float32, numpy=
array([[[4.4560995e+00, 2.6941299e-03],
        [8.5883799e+00, 6.0469198e+00]],

       [[3.6264551e+00, 6.6117930e+00],
        [5.9046197e+00, 6.6545572e+00]]], dtype=float32)>